In [176]:
# # import all modules
import shutil
import glob
import csv
import ast
import os
import re

import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from matplotlib.lines import Line2D


import pandas as pd
import numpy as np
import math
import pymatgen.core as mg
import pymatgen.symmetry.analyzer as amg



# Calculate Rotational Symmetry Number with pymatgen

In [195]:
def get_symmetry_number(file):
    mol = amg.Molecule.from_file(file)
    return amg.PointGroupAnalyzer(mol).get_rotational_symmetry_number()

# Add column for Rotor Type to Groundstate file

In [197]:
df_ground = pd.read_csv('/Users/z5380625/Documents/Research/4_BigData/B971_def2TZVPD_DataFile1_GroundState.csv') # load groundstate csv file 
rays_asymm = (2*df_ground['B [cm-1]'].values - df_ground['A [cm-1]'].values - df_ground['C [cm-1]'].values)/(df_ground['A [cm-1]'].values - df_ground['C [cm-1]'].values)
df_ground['rays_asymm'] = rays_asymm
df_ground

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Formula,SMILES,# Atoms,Conf#,Rotamer Degeneracy,Relative Energy [kJ/mol],BW Percent at 10 K,...,mu_c [D],mu_tot [D],mu_amu_bmu_c,A [cm-1],B [cm-1],C [cm-1],Flag1,Flag2,Flag3,rays_asymm
0,0,0,0,C2H2,C#C,4,CONF1,1,0.0,100.0,...,-0.000005,0.000005,-1.666957e-22,0.000000,1.184735,1.184735,Conformer Retained,Present,Complete,-1.000000
1,1,1,1,C2H2O2,O=CC=O,6,CONF1,1,0.0,100.0,...,0.000007,0.000007,-1.446818e-17,1.861842,0.159363,0.146798,Conformer Retained,Present,Complete,-0.985347
2,2,2,2,C2H2OS_1,C#CSO,6,CONF1,1,0.0,100.0,...,-1.353319,1.571004,4.299470e-01,0.672851,0.131892,0.111405,Conformer Retained,Present,Complete,-0.927021
3,3,3,3,C2H2OS_2,O=CC=S,6,CONF1,1,0.0,100.0,...,0.000095,1.122373,-9.414538e-06,1.725695,0.095213,0.090234,Conformer Retained,Present,Complete,-0.993912
4,4,4,4,C2H2O_1,C1=CO1,5,CONF1,1,0.0,100.0,...,-0.000187,2.331551,-2.615070e-05,1.221050,0.800449,0.483497,Conformer Retained,Present,Complete,-0.140530
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2738,10196,10196,10196,C6H8_5,CC(C1)=C1C=C,14,CONF1,3,0.0,100.0,...,0.000238,0.940538,1.039541e-04,0.317003,0.068717,0.057732,Conformer Retained,Present,Complete,-0.915260
2739,10198,10198,10198,C6H8_50,C=C1CC12CC2,14,CONF1,2,0.0,100.0,...,0.000017,0.671013,1.922556e-06,0.238074,0.092912,0.081065,Conformer Retained,Present,Complete,-0.849102
2740,10200,10200,10200,C6H8_6,CC1=CC1C=C,14,CONF1,3,0.0,100.0,...,-0.133619,1.459553,3.979669e-02,0.276880,0.070069,0.060904,Conformer Retained,Present,Complete,-0.915128
2741,10201,10201,10201,C6H8_8,CC1=CC=CC1,14,CONF1,3,0.0,100.0,...,0.000000,0.740155,0.000000e+00,0.268572,0.107810,0.079179,Conformer Retained,Present,Complete,-0.697651


In [200]:
formulas = df_ground['Formula'].values
num_atoms = df_ground['# Atoms'].values
confs = df_ground['Conf#'].values

formula_conf_list = list(zip(formulas,confs,num_atoms))
sigma_list = []
for formula_conf in formula_conf_list:
    formula = formula_conf[0]
    conf = formula_conf[1]
    num = formula_conf[2]
    if num <=6:
        folder_loc ='6_Atoms'
    elif (num>6) & (num<=8):
        folder_loc = '8_Atoms'
    else:
        folder_loc = str(num)+'_Atoms'
    file = '/Users/z5380625/Documents/Research/4_BigData/'+folder_loc+'/4_VibrationalCalcs/FinalGeoms_VibCalc/'+formula+'_'+conf+'_VibCalcGeom.xyz'
    sigma = get_symmetry_number(file)
    sigma_list.append(sigma)
    


In [203]:
df_ground['Sigma'] = sigma_list

In [204]:
A=df_ground['A [cm-1]'].values
B=df_ground['B [cm-1]'].values
C=df_ground['C [cm-1]'].values
sigma=df_ground['Sigma'].values
ABC_list = list(zip(A,B,C,sigma))

rotor_type_list = []
qrot_list = []
for ABC in ABC_list:
    A = ABC[0]*29979.2458
    B = ABC[1]*29979.2458
    C = ABC[2]*29979.2458
    sigma = ABC[3]
    rotor_type = define_mol_rotor_type(A,B,C)
    if rotor_type == 'Spherical':
        rotor_type_list.append('Spherical')
        qrot = qrot_asym(A,B,C,sigma)
        qrot_list.append(qrot)
    elif rotor_type == 'Linear':
        rotor_type_list.append('Linear')
        qrot = qrot_linear(B)
        qrot_list.append(qrot)
    elif rotor_type == 'Prolate':
        rotor_type_list.append('Prolate')
        qrot = qrot_prolate(A,B,sigma)
        qrot_list.append(qrot)
    elif rotor_type == 'Oblate':
        rotor_type_list.append('Oblate')
        qrot = qrot_oblate(B,C,sigma)
        qrot_list.append(qrot)
    elif rotor_type == 'Asymmetric':
        rotor_type_list.append('Asymmetric')
        qrot = qrot_asym(A,B,C,sigma)
        qrot_list.append(qrot)


df_ground['Rotor Type'] = rotor_type_list
df_ground['Qrot @ 9.375 K'] = qrot_list


# Define Molecular Rotor Type

In [170]:
def define_mol_rotor_type(A,B,C):
    # spherical
    if math.isclose(A,B,rel_tol=1e-3) and math.isclose(B,C,rel_tol=1e-3):
        return 'Spherical'
    # linear
    elif math.isclose(A,0,rel_tol=1e-3) and math.isclose(B,C,rel_tol=1e-3):
        return 'Linear'
    # prolate
    elif math.isclose(B,C,rel_tol=1e-3) and A>B:
        return 'Prolate'
    # oblate
    elif math.isclose(A,B,rel_tol=1e-3) and B>C:
        return 'Oblate'
    # asymmetric
    else:
        return 'Asymmetric'
    

# Calculate Qrot at 9.375 K for all Molecules

In [168]:
def qrot_linear(B):
    return 2.0837e4*(9.375/B)

def qrot_prolate(A,B,sigma):
    return 5.3311e6*(9.375**3/(B**2*A))**0.5/sigma

def qrot_oblate(B,C,sigma):
    return 5.3311e6*(9.375**3/(B**2*C))**0.5/sigma

def qrot_asym(A,B,C,sigma):
    return 5.3311e6*(9.375**3/(A*B*C))**0.5/sigma

# Create .var files for SPCAT

line 1: title 
line 2 [freeform]: NPAR, NLINE, NITR, NXPAR, THRESH , ERRTST, FRAC, CAL
(only NPAR used by SPCAT)
line 3 option information[freeform]:
CHR, SPIND, NVIB, KNMIN, KNMAX, IXX, IAX, WTPL, WTMN, VSYM, EWT, DIAG, XOPT

In [211]:
# file_names = glob.glob(comp_loc+'4_BigData/'+folder_loc+'/4_VibrationalCalcs/OutputFiles/*.log')
def make_spcat_var(file_loc,file):
    filen = file.split('/')[-1].split('_harmonic')[0]
    with open(file,'r') as inp:
        lines = inp.readlines()
        indices = [i for i, x in enumerate(lines) if x == '--------------------------------------------------------------------------------\n']
        spcat_var_list = lines[indices[-2]+1:indices[-1]]

    inp.close()
    
    spcat_var_list[2] = 'a 00 1 0 99\n'

    print(spcat_var_list)


    with open('/Users/z5380625/Documents/Research/4_BigData/'+folder_loc+'/4_VibrationalCalcs/InputFiles_SPCAT/'+filen+'.var','a') as f:
        for line in spcat_var_list:
            f.write(line)
    f.close()
    return
            

# Create .int files for SPCAT


line 1:title
line 2 [freeform]: FLAGS,TAG,QROT,FBGN,FEND,STR0,STR1,FQLIM,TEMP,MAXV
line 3-end [freeform]: IDIP,DIPOLE

In [239]:
def get_dipole_moments(file):
    dipole_info_all = []
    with open(file,'r+') as inp:
        for line in inp:
            line = line.strip()
            if line.startswith('Dipole moment (Debye):'):
                # print(next(inp).split())
                dipole_info_all.append(next(inp).split())
    inp.close()
    row = dipole_info_all[-1]
    mu_a, mu_b, mu_c = float(row[0]),float(row[1]),float(row[2])
    return mu_a, mu_b, mu_c


In [238]:


file = '/Users/z5380625/Documents/Research/4_BigData/6_Atoms/4_VibrationalCalcs/OutputFiles/H3P_CONF1_harmonic_conformers.log'
folder_loc = '6_Atoms'

i=1
for file in files_list:
    mol = file.split('/')[-1].split('_harmonic')[0].split('_')[0]
    filen = file.split('/')[-1].split('_harmonic')[0]
    identifier = '{0:0>4}'.format(i)
    qrot = '{0:0.3}'.format(df_ground[df_ground['Formula']==mol]['Qrot @ 9.375 K'].values[0])
    mu_a, mu_b, mu_c = get_dipole_moments(file)


    print('{0:.7f}'.format(mu_b))
    with open('/Users/z5380625/Documents/Research/4_BigData/'+folder_loc+'/4_VibrationalCalcs/InputFiles_SPCAT/'+filen+'.int','a') as f:
        f.write(filen+'\n')
        f.write('112    '+identifier+'  '+qrot+'    0   20  -8. -8. 30000   9.375\n')
        f.write('        001    '+'{0:.7f}'.format(mu_a)+'                / a dipole /\n')
        f.write('        002    '+'{0:.7f}'.format(mu_b)+'                / b dipole /\n')
        f.write('        003    '+'{0:.7f}'.format(mu_c)+'                / c dipole /\n')

    f.close()


-0.0000227
